In [1]:
class Descriptor:
    
    def __get__(self, obj, obj_type):
        print('get')
        
    def __set__(self, obj, value):
        print('set')
    
    def __delete__(self, obj):
        print('delete')

class Class:
    # Атрибут будет являться дескриптором, 
    # т.е. переопределено поведение к доступу, присваивании и удалении атрибута.
    attr = Descriptor()

instance = Class()

In [2]:
instance.attr

get


In [3]:
instance.attr = 10

set


In [4]:
del instance.attr

delete


In [5]:
class Value:
    
    def __init__(self):
        self.value = None
    
    @staticmethod
    def _prepare_value(value):
        return value * 10
    
    def __get__(self, obj, obj_type): # obj=instance, obj_type=Class
        return self.value
        
    def __set__(self, obj, value):
        self.value = self._prepare_value(value)
        
class Class:
    attr = Value()
    

instance = Class()
instance.attr = 10

print(instance.attr)

100


#### Написать дескриптор, который пишет в файл все присваемые ему значения. 

In [26]:
class ImportantValue:
    
    def __init__(self, amount):
        self.amount = amount
    
    def __get__(self, obj, obj_type):
        return self.amount
    
    def __set__(self, obj, value):
        with open('log.txt', 'a') as f:
            f.write(str(value) + '\n')
        
        self.amount = value
    

class Account:
    amount = ImportantValue(100)
    
bobs_account = Account()
bobs_account.amount = 200

with open('log.txt', 'r') as f:
    print(f.read())

150200200
200200200
200
200



### Функции и методы

In [19]:
class Class:
    def method(self):
        pass

obj = Class()

print(obj.method)   # Bound метод
print(Class.method)    # Unbound метод
# В bound метод по умолчанию передаётся объект, с которым вызван метод.
# он и записывается в атрибут self метода класса.

<bound method Class.method of <__main__.Class object at 0x7f7d74890940>>
<function Class.method at 0x7f7d740a4620>


#### @property

In [27]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
amy = User('Amy', 'Jones')

print(amy.full_name)
print(User.full_name)

Amy Jones


In [28]:
# Создадим свой property
class Property:
    def __init__(self, getter):
        self.getter = getter
        
    def __get__(self, obj, obj_type=None):
        if obj is None:
            return self
        return self.getter(obj)

In [29]:
class Class:
    @property
    def original(self):
        return 'original'
    
    @Property
    def custom_sugar(self):
        return 'custom_sugar'
    
    def custom_pure(self):
        return 'custon pure'
    
    custom_pure = Property(custom_pure)

In [31]:
obj = Class()

print(obj.original)
print(obj.custom_sugar)
print(obj.custom_pure)

original
custom_sugar
custon pure


#### StaticMethod and ClassMethod

In [32]:
class StaticMethod:
    def __init__(self, func):
        self.func = func
    def __get__(self, obj, obj_type=None):
        return self.func

In [33]:
class ClassMethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, obj, obj_type=None):
        if obj_type is None:
            obj_type = type(obj)
            
        def new_func(*args, **kwargs):
            return self.func(obj_type, *args, **kwargs)
        
        return new_func

#### __slots__

In [34]:
class Class:
    
    __slots__ = ['anakin']    # Определить класс с жестко заданным набором аттрибутов.
    # При создании класса у него создается словарь в кот. мы записываем атрибуты, которые добавляются в объект.
    def __init__(self):
        self.anakin = 'the chosen one'

obj = Class()

obj.luke = 'the chosen too'

AttributeError: 'Class' object has no attribute 'luke'